In [1]:
from SPARQLWrapper import SPARQLWrapper, JSON

In [2]:
def get_all_ids_from_object(url):
    objs = []

    sparql = SPARQLWrapper("http://rdf.insee.fr/sparql")
    query = """
    SELECT * WHERE {
     ?s a <"""+url+""">
    }
    """
    sparql.setQuery(query)
    sparql.setReturnFormat(JSON)
    results = sparql.query().convert()

    for result in results["results"]["bindings"]:
        objs.append(result['s']['value'])
    print('Nombre de {} : {}'.format(url.replace('http://rdf.insee.fr/def/geo#',''),len(objs)))
    return objs


In [3]:
coms = get_all_ids_from_object("http://rdf.insee.fr/def/geo#Commune")
comds = get_all_ids_from_object("http://rdf.insee.fr/def/geo#CommuneAssociee")
comas = get_all_ids_from_object("http://rdf.insee.fr/def/geo#CommuneDeleguee")
comfs = get_all_ids_from_object("http://rdf.insee.fr/def/geo#CommuneFusionee")
evt = get_all_ids_from_object("http://rdf.insee.fr/def/geo#EvenementGeographique")


Nombre de Commune : 43287
Nombre de CommuneAssociee : 1046
Nombre de CommuneDeleguee : 2449
Nombre de CommuneFusionee : 0
Nombre de EvenementGeographique : 5086


In [4]:
def get_info_com(typeCom, comArray):
    print("-----"+typeCom+"-----")
    i = 0
    for com in comArray:
        if(i%100 == 0):
            print(i)
        i = i + 1
        sparql = SPARQLWrapper("http://rdf.insee.fr/sparql")
        sparql.setQuery("""
           DESCRIBE <{}>
        """.format(com))
        sparql.setReturnFormat(JSON)
        results = sparql.query().convert()
        newcom = {}

        for result in results["results"]["bindings"]:
            if(result['subject']['value'] == com):
                if(result['predicate']['value'] == 'http://rdf.insee.fr/def/geo#codeINSEE'):
                    newcom['codeInsee'] = result['object']['value']
                if(result['predicate']['value'] == 'http://rdf.insee.fr/def/geo#nom'):
                    newcom['nom'] = result['object']['value']
                if(result['predicate']['value'] == 'http://rdf.insee.fr/def/geo#nomSansArticle'):
                    newcom['nomSansArticle'] = result['object']['value']
            if(result['object']['value'] == com):
                if(result['predicate']['value'] == 'http://rdf.insee.fr/def/geo#creation'):
                    newcom['evtCreation'] = result['subject']['value'].replace('http://id.insee.fr/geo/evenementGeographique/','')
                if(result['predicate']['value'] == 'http://rdf.insee.fr/def/geo#suppression'):
                    newcom['evtSuppression'] = result['subject']['value'].replace('http://id.insee.fr/geo/evenementGeographique/','')

        newcom['id'] = com.replace('http://id.insee.fr/geo/commune/','').replace('http://id.insee.fr/geo/communeAssociee/','').replace('http://id.insee.fr/geo/communeDeleguee/','').replace('http://id.insee.fr/geo/communeFusionnee/','')
        newcom['type'] = typeCom

        listallcoms.append(newcom)

Récupération du détail de toutes les communes (de tout type COMA COMD COM COMF) dans un tableau

In [5]:
listallcoms = []
get_info_com('COM',coms)
get_info_com('COMA',comas)
get_info_com('COMD',comds)
get_info_com('COMF',comfs)

-----COM-----
0


/home/thomasg/git/cog-sparql-analysis/venv/lib/python3.9/site-packages/SPARQLWrapper/Wrapper.py:979: RuntimeWarning: Sending Accept header '*/*' because unexpected returned format 'json' in a 'DESCRIBE' SPARQL query form
  warnings.warn("Sending Accept header '*/*' because unexpected returned format '%s' in a '%s' SPARQL query form" % (self.returnFormat, self.queryType), RuntimeWarning)


100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100
5200
5300
5400
5500
5600
5700
5800
5900
6000
6100
6200
6300
6400
6500
6600
6700
6800
6900
7000
7100
7200
7300
7400
7500
7600
7700
7800
7900
8000
8100
8200
8300
8400
8500
8600
8700
8800
8900
9000
9100
9200
9300
9400
9500
9600
9700
9800
9900
10000
10100
10200
10300
10400
10500
10600
10700
10800
10900
11000
11100
11200
11300
11400
11500
11600
11700
11800
11900
12000
12100
12200
12300
12400
12500
12600
12700
12800
12900
13000
13100
13200
13300
13400
13500
13600
13700
13800
13900
14000
14100
14200
14300
14400
14500
14600
14700
14800
14900
15000
15100
15200
15300
15400
15500
15600
15700
15800
15900
16000
16100
16200
16300
16400
16500
16600
16700
16800
16900
17000
17100
17200
17300
17400
17500
17600
17700
17800
17900
18000
18100
18200
18300
18400
1850

Sauvegarde dans un fichier local (pour éviter de rejouer ce code à chaque fois)

In [10]:
import json
with open('coms.json', 'w') as f:
    json.dump(listallcoms, f, indent=4)

Récupération du détail de tous les événements dans un tableau

In [7]:
listallevt = []
i = 0
for e in evt:
    if(i%100 == 0):
        print(i)
    i = i + 1
    sparql = SPARQLWrapper("http://rdf.insee.fr/sparql")
    sparql.setQuery("""
       DESCRIBE <{}>
    """.format(e))
    sparql.setReturnFormat(JSON)
    results = sparql.query().convert()
    newevt = {}
    newevt['creation'] = []
    newevt['suppression'] = []
    
    for result in results["results"]["bindings"]:
        if(result['predicate']['value'] == 'http://rdf.insee.fr/def/geo#creation'):
            newevt['creation'].append(result['object']['value'].split('/')[-1])
        if(result['predicate']['value'] == 'http://rdf.insee.fr/def/geo#suppression'):
            newevt['suppression'].append(result['object']['value'].split('/')[-1])
        if(result['predicate']['value'] == 'http://rdf.insee.fr/def/geo#date'):
            newevt['date'] = result['object']['value']
                
    newevt['id'] = e.replace('http://id.insee.fr/geo/evenementGeographique/','')
    newevt['type'] = 'evenement'

    listallevt.append(newevt)

0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000


Sauvegarde dans un fichier json pour éviter de rejouer le traitement

In [9]:
import json
with open('evts.json', 'w') as f:
    json.dump(listallevt, f, indent=4)